In [195]:
import numpy as np
import pandas as pd
import matplotlib as plt
import plotly.graph_objects as go

In [461]:
df = pd.read_csv("world-electricity-by-source.csv")
not_countries = ["European Union","Euro area","Sub-Saharan Africa (excluding high income)","Sub-Saharan Africa (IDA & IBRD)", "High income", "Middle East & North Africa (IDA & IBRD)","Sub-Saharan Africa","OECD members","Central Europe and the Baltics","Low income","Post-demographic dividend","IDA blend","Middle East & North Africa (excluding high income)","Middle East & North Africa","South Asia (IDA & IBRD)","Middle East & North Africa (IDA & IBRD)","IDA total","IDA only","Latin America & Caribbean (excluding high income)","Latin America & Caribbean","Latin America & Caribbean (IDA & IBRD)","Small states","Other small states","Early-demographic dividend","East Asia & Pacific (IDA & IBRD)","World", "East Asia & Pacific (excluding high income)","IDA & IBRD total","IBRD only","South Asia (IDA & IBRD","Upper middle income","Low & middle income","Middle income","Late-demographic dividend","East Asia & Pacific","Lower middle income"]
df.columns = ['Entity', 'Code', 'Year', 'Coal', 'Nuclear','Natural_gas', 'Oil', 'Hydroelectric', 'Other_renewables']
df.drop(columns=['Code'], inplace=True)

df_1971_2015 = df[(df["Year"] == 1975) | (df["Year"] == 2015)]
df_unstack = df_1971_2015.set_index(['Entity', 'Year']).unstack()
df_unstack

df_unstack["Difference", "Coal"] = df_unstack["Coal"][2015] - df_unstack["Coal"][1975] 
df_unstack["Difference", "Nuclear"] = df_unstack["Nuclear"][2015] - df_unstack["Nuclear"][1975] 
df_unstack["Difference", "Natural_gas"] = df_unstack["Natural_gas"][2015] - df_unstack["Natural_gas"][1975] 
df_unstack["Difference", "Oil"] = df_unstack["Oil"][2015] - df_unstack["Oil"][1975] 
df_unstack["Difference", "Hydroelectric"] = df_unstack["Hydroelectric"][2015] - df_unstack["Hydroelectric"][1975] 
df_unstack["Difference", "Other_renewables"] = df_unstack["Other_renewables"][2015] - df_unstack["Other_renewables"][1975] 

df_difference = df_unstack.drop(columns=["Coal", "Nuclear", "Natural_gas", "Oil", "Hydroelectric", "Other_renewables"])
df_difference.columns = df_difference.columns.droplevel()
df_difference.columns.name = ""
df_difference.sort_values(by="Coal", inplace=True)
df_difference.fillna(0, inplace=True)
df_difference.drop(not_countries, axis=0, errors="ignore", inplace=True)
df_difference.head(10)

,Coal,Nuclear,Natural_gas,Oil,Hydroelectric,Other_renewables
Entity,,,,,,
Vietnam,-49.835473,0.000000,33.210467,0.486029,16.020895,0.118082
Luxembourg,-46.679317,0.000000,39.794379,-23.719165,0.701774,24.943651
Slovakia,-39.975832,56.758904,-0.731602,-22.942896,-0.458571,8.163112
United Kingdom,-39.776323,9.778770,26.361214,-20.815313,0.484539,22.970305
Hungary,-37.005780,52.446093,-2.083772,-23.554355,-0.025001,9.811482
Czech Republic,-28.212266,32.490831,2.110869,-14.031903,-1.778678,9.231868
Finland,-23.357429,33.898478,4.336811,-18.360468,-21.987399,20.053355
North Korea,-22.583273,0.000000,0.000000,3.339088,19.244186,0.000000
Germany,-19.011808,7.927832,-7.101247,-7.280259,-1.482864,25.983676


## Decrease coal

In [472]:
df_decrease_coal = df_difference[df_difference["Coal"] < -5]
df_decrease_coal.drop(columns=["Coal"], inplace=True)
df_decrease_coal[df_decrease_coal < 0] = 0
df_decrease_coal = df_decrease_coal.apply(lambda x: round(x/x.sum(),2), axis=1)
df_decrease_coal["New_source"] = df_decrease_coal.idxmax(axis=1)
df_decrease_coal = df_decrease_coal.reset_index().sort_values("New_source").set_index(["New_source", "Entity"])

df_decrease_count = df_decrease_coal.reset_index().groupby("New_source").count()["Entity"].sort_values(ascending=False)

fig = go.Figure([go.Bar(x=df_decrease_count.index, y=df_decrease_count.values)])
fig.show()

/home/xavier/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Increase coal

In [470]:
df_increase_coal = df_difference[df_difference["Coal"] > 5]
df_increase_coal.drop(columns=["Coal"], inplace=True)
df_increase_coal[df_increase_coal > 0] = 0
df_increase_coal = df_increase_coal.apply(lambda x: round(x/x.sum(),2), axis=1)
df_increase_coal["Old_source"] = df_increase_coal.idxmax(axis=1)
df_increase_coal = df_increase_coal.reset_index().sort_values("Old_source").set_index(["Old_source", "Entity"])
df_increase_count = df_increase_coal.reset_index().groupby("Old_source").count()["Entity"].sort_values(ascending=False)

fig = go.Figure([go.Bar(x=df_increase_count.index, y=df_increase_count.values)])
fig.show()

/home/xavier/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

